<a href="https://colab.research.google.com/github/avijit-mukherjee-25/llm/blob/main/BERT_finetune_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install libraries
!pip install transformers torch numpy datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [2]:
!pip install peft

In [3]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=d60730f187dc6775e8f50fe993bbd7c4706ed0edb784a30f8b038018d3a7d931
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


In [4]:
import torch, datasets
import torch

In [5]:
# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


## Get Data

In [6]:
import wget
import os

print('Downloading dataset...')

# The URL for the dataset zip file.
url = 'https://nyu-mll.github.io/CoLA/cola_public_1.1.zip'

# Download the file (if we haven't already)
if not os.path.exists('./cola_public_1.1.zip'):
    wget.download(url, './cola_public_1.1.zip')

In [7]:
# Unzip the dataset (if we haven't already)
if not os.path.exists('./cola_public/'):
    !unzip cola_public_1.1.zip

Archive:  cola_public_1.1.zip
   creating: cola_public/
  inflating: cola_public/README      
   creating: cola_public/tokenized/
  inflating: cola_public/tokenized/in_domain_dev.tsv  
  inflating: cola_public/tokenized/in_domain_train.tsv  
  inflating: cola_public/tokenized/out_of_domain_dev.tsv  
   creating: cola_public/raw/
  inflating: cola_public/raw/in_domain_dev.tsv  
  inflating: cola_public/raw/in_domain_train.tsv  
  inflating: cola_public/raw/out_of_domain_dev.tsv  


In [8]:
import pandas as pd

# Load the dataset into a pandas dataframe.
df = pd.read_csv("./cola_public/raw/in_domain_train.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])
df = df[['label','sentence']].copy()

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

# Display 10 random rows from the data.
df.sample(10)

Number of training sentences: 8,551



,label,sentence
1669,1,What Tom said Al thought you wanted me to make...
7633,1,Several people seem several people sick.
5079,1,It was to the student that the teacher gave th...
2621,0,Doug removed the tabletop of scratches.
6742,1,Because of the strike the commuters travelled ...
4445,0,George has went to America.
6361,1,John talked to a woman.
4932,1,He made a statement everyone thought was inter...
2513,0,The teacher hated the pupils angry.
210,1,"The more people that arrive, the louder it gets."


In [9]:
df.shape

(8551, 2)

In [10]:
import numpy as np
np.random.seed(1337)
idx = np.random.random(size=df.shape[0])<0.8
df_train = df.iloc[idx]
df_test = df.iloc[~idx]
print (df_train.shape, df_test.shape)

(6836, 2) (1715, 2)


## Prepare Data

Approach 1

In [11]:
from transformers import AutoTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased", do_lower_case=True)

# # Alternatively, we could have used BertTokenizer directly
# from transformers import BertTokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [12]:
# Print the original sentence.
sentences = df.sentence.values
labels = df.label.values

print(' Original: ', sentences[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(sentences[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

# encode
print (tokenizer.encode(sentences[0], add_special_tokens=True))

# special tokens
print (tokenizer.convert_ids_to_tokens(101), tokenizer.convert_ids_to_tokens(102))

print (
    tokenizer.encode_plus(
        sentences[0],
        add_special_tokens=True,
        max_length=64,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


 Original:  Our friends won't buy this analysis, let alone the next one we propose.
Tokenized:  ['our', 'friends', 'won', "'", 't', 'buy', 'this', 'analysis', ',', 'let', 'alone', 'the', 'next', 'one', 'we', 'propose', '.']
Token IDs:  [2256, 2814, 2180, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012]
[101, 2256, 2814, 2180, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012, 102]
[CLS] [SEP]
{'input_ids': tensor([[  101,  2256,  2814,  2180,  1005,  1056,  4965,  2023,  4106,  1010,
          2292,  2894,  1996,  2279,  2028,  2057, 16599,  1012,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0]]), 'token

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2681: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [13]:
max_len = 0

# For every sentence...
for sent in sentences:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  47


In [14]:
# set max_len to 64
max_len = 64
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = max_len,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )

    # Add the encoded sentence to the list.
    input_ids.append(encoded_dict['input_ids'])

    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Original:  Our friends won't buy this analysis, let alone the next one we propose.
Token IDs: tensor([  101,  2256,  2814,  2180,  1005,  1056,  4965,  2023,  4106,  1010,
         2292,  2894,  1996,  2279,  2028,  2057, 16599,  1012,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])


## Prepare Data

Approach 2 (Using Transformers Dataset class)

In [15]:
from datasets import Dataset
train_data = Dataset.from_pandas(df_train)
# # alternatively we could have using from_dict as follows
# data = Dataset.from_dict({"sentence": sentences, "label": labels})
train_data

Dataset({
    features: ['label', 'sentence', '__index_level_0__'],
    num_rows: 6836
})

In [16]:
test_data = Dataset.from_pandas(df_test)
test_data

Dataset({
    features: ['label', 'sentence', '__index_level_0__'],
    num_rows: 1715
})

In [17]:
train_data[0], test_data[0]

({'label': 1,
  'sentence': "Our friends won't buy this analysis, let alone the next one we propose.",
  '__index_level_0__': 0},
 {'label': 1,
  'sentence': 'Bill coughed his way out of the restaurant.',
  '__index_level_0__': 7})

In [18]:
def tokenize_function(_data):
    return tokenizer(_data['sentence'], padding="max_length", truncation=True)

In [19]:
train_data = train_data.map(tokenize_function, batched=True)
test_data = test_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/6836 [00:00<?, ? examples/s]

Map:   0%|          | 0/1715 [00:00<?, ? examples/s]

In [20]:
train_data = train_data.rename_columns({"label": "labels"})
test_data = test_data.rename_column("label", "labels")
train_data, test_data

(Dataset({
     features: ['labels', 'sentence', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 6836
 }),
 Dataset({
     features: ['labels', 'sentence', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 1715
 }))

## Train using Pytorch Trainer

In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-uncased", num_labels=2, torch_dtype="auto")
model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
import numpy as np
import evaluate

In [ ]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments(
    output_dir='test_trainer',
    eval_strategy="epoch",
    num_train_epochs=2,
    report_to="none"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    compute_metrics=compute_metrics,
)

In [ ]:
# trainer.train()

## Finetune using native Pytorch

In [ ]:
from accelerate.utils.memory import clear_device_cache
del model
del trainer
clear_device_cache()

In [ ]:
train_data.set_format('torch')
test_data.set_format('torch')

In [ ]:
# create dataloader
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=32)
test_dataloader = DataLoader(test_data, shuffle=True, batch_size=32)

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-uncased", num_labels=2)
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=1e-4)

In [ ]:
from transformers import get_scheduler
num_epochs = 2
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
def train(model, optimizer, lr_scheduler, train_dataloader):
    train_loss = 0.0
    model.train()
    for step, batch in enumerate(train_dataloader):
        input_ids = batch['input_ids'].to(device)
        input_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        result = model(input_ids,
               token_type_ids=None,
               attention_mask=input_mask,
               labels=labels,
               return_dict=True)
        loss = result.loss
        logits = result.logits
        train_loss += loss.item()

        loss.backward()
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        if step%100==0:
            print (f'training loss at {step} is {train_loss}')
    return train_loss

In [ ]:
import evaluate
metric = evaluate.load("accuracy")

@torch.no_grad()
def eval(model, test_dataloader):
    model.eval()
    val_loss = 0.0
    for _, batch in enumerate(test_dataloader):
        input_ids = batch['input_ids'].to(device)
        input_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        result = model(input_ids,
               token_type_ids=None,
               attention_mask=input_mask,
               labels=labels,
               return_dict=True)
        loss = result.loss
        val_loss += loss.item()
        logits = result.logits
        predictions = torch.argmax(logits, dim=-1)
        metric.add_batch(predictions=predictions, references=labels)
    print (f'validation loss is {val_loss}')
    val_accuracy = metric.compute()
    model.train()
    return val_accuracy

In [ ]:
# for epoch in range(num_epochs):
#     train_loss = train(model, optimizer, lr_scheduler, train_dataloader)
#     print (f'train loss at epoch {epoch} is {train_loss}')
#     val_accuracy = eval(model, test_dataloader)
#     print (f'validation accuracy at epoch {epoch} is {val_accuracy}')

training loss at 0 is 0.6846567392349243
training loss at 100 is 55.85948145389557
training loss at 200 is 105.27676033973694
train loss at epoch 0 is 111.08661270141602
validation loss is 25.569703608751297
validation accuracy at epoch 0 is {'accuracy': 0.7784256559766763}
training loss at 0 is 0.32113173604011536
training loss at 100 is 25.013138458132744
training loss at 200 is 49.57106313854456
train loss at epoch 1 is 52.846872825175524
validation loss is 25.306149199604988
validation accuracy at epoch 1 is {'accuracy': 0.8122448979591836}


## Finetune (native PyTorch) BERT using LoRA

In [22]:
# from accelerate.utils.memory import clear_device_cache
# del model
# del trainer
# clear_device_cache()

In [23]:
train_data.set_format('torch')
test_data.set_format('torch')

In [24]:
# create dataloader
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=32)
test_dataloader = DataLoader(test_data, shuffle=True, batch_size=32)

In [25]:
from peft import LoraConfig, TaskType,  get_peft_config, get_peft_model

In [26]:
peft_config = LoraConfig(task_type=TaskType.SEQ_CLS, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)

In [27]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-uncased", num_labels=2, torch_dtype="auto")
model = get_peft_model(model, peft_config)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
model.print_trainable_parameters()

trainable params: 296,450 || all params: 109,780,228 || trainable%: 0.2700


In [29]:
model.to(device)

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default

In [30]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)

In [31]:
from transformers import get_scheduler
num_epochs = 2
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [32]:
def train(model, optimizer, lr_scheduler, train_dataloader):
    train_loss = 0.0
    model.train()
    for step, batch in enumerate(train_dataloader):
        input_ids = batch['input_ids'].to(device)
        input_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        result = model(input_ids,
               token_type_ids=None,
               attention_mask=input_mask,
               labels=labels,
               return_dict=True)
        loss = result.loss
        logits = result.logits
        train_loss += loss.item()

        loss.backward()
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        if step%100==0:
            print (f'training loss at {step} is {train_loss}')
    return train_loss

In [33]:
import evaluate
metric = evaluate.load("accuracy")

@torch.no_grad()
def eval(model, test_dataloader):
    model.eval()
    val_loss = 0.0
    for _, batch in enumerate(test_dataloader):
        input_ids = batch['input_ids'].to(device)
        input_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        result = model(input_ids,
               token_type_ids=None,
               attention_mask=input_mask,
               labels=labels,
               return_dict=True)
        loss = result.loss
        val_loss += loss.item()
        logits = result.logits
        predictions = torch.argmax(logits, dim=-1)
        metric.add_batch(predictions=predictions, references=labels)
    print (f'validation loss is {val_loss}')
    val_accuracy = metric.compute()
    model.train()
    return val_accuracy

In [34]:
for epoch in range(num_epochs):
    train_loss = train(model, optimizer, lr_scheduler, train_dataloader)
    print (f'train loss at epoch {epoch} is {train_loss}')
    val_accuracy = eval(model, test_dataloader)
    print (f'validation accuracy at epoch {epoch} is {val_accuracy}')

training loss at 0 is 0.9069423675537109
training loss at 100 is 64.52657976746559
training loss at 200 is 124.33152765035629
train loss at epoch 0 is 131.3740637600422
validation loss is 32.56726595759392
validation accuracy at epoch 0 is {'accuracy': 0.6944606413994169}
training loss at 0 is 0.5476846694946289
training loss at 100 is 60.66928189992905
training loss at 200 is 117.12469211220741
train loss at epoch 1 is 124.35750171542168
validation loss is 31.97016641497612
validation accuracy at epoch 1 is {'accuracy': 0.6944606413994169}


## Finetune BERT-LoRA using Trainer

In [37]:
# from accelerate.utils.memory import clear_device_cache
# del model
# del trainer
# clear_device_cache()

In [38]:
train_data.set_format('torch')
test_data.set_format('torch')

In [39]:
from transformers import AutoModelForSequenceClassification

In [40]:
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-uncased", num_labels=2, torch_dtype="auto")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
from peft import LoraConfig, TaskType, get_peft_model

In [42]:
peft_config = LoraConfig(task_type=TaskType.SEQ_CLS, inference_mode=False, r=8, lora_dropout=0.1)

In [43]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 296,450 || all params: 109,780,228 || trainable%: 0.2700


In [44]:
from transformers import TrainingArguments, Trainer

In [45]:
import numpy as np
import evaluate

In [46]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [47]:
training_args = TrainingArguments(
    output_dir='test_trainer',
    eval_strategy="epoch",
    num_train_epochs=2,
    report_to="none",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
)

In [48]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    compute_metrics=compute_metrics,
)

In [50]:
# trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.609842,0.694461
2,No log,0.604416,0.694461


TrainOutput(global_step=428, training_loss=0.5995607465227074, metrics={'train_runtime': 1044.5708, 'train_samples_per_second': 13.089, 'train_steps_per_second': 0.41, 'total_flos': 3609705362718720.0, 'train_loss': 0.5995607465227074, 'epoch': 2.0})